In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.1.2


In [15]:
import glob

In [2]:
data_path = "/home/ec2-user/SageMaker/DeHurricane/data_downloaded/TCIR-ATLN_EPAC_WPAC.h5"


In [3]:
data_info = pd.read_hdf(data_path, key="info", mode='r')


In [4]:
hf=h5py.File(data_path, 'r')

In [5]:
data_matrix = hf['matrix']

In [8]:
data_matrix

<HDF5 dataset "matrix": shape (47381, 201, 201, 4), type "<f4">

In [12]:
data_info['Vmax'].values[i]

95.0

In [31]:
data_matrix[i][:, :, 0::3]

array([[[2.68399994e+02, 0.00000000e+00],
        [2.67279999e+02, 0.00000000e+00],
        [2.65139984e+02, 0.00000000e+00],
        ...,
        [2.90429993e+02, 0.00000000e+00],
        [2.90750000e+02, 0.00000000e+00],
        [2.91290009e+02, 0.00000000e+00]],

       [[2.68209991e+02, 0.00000000e+00],
        [2.68209991e+02, 0.00000000e+00],
        [2.65589996e+02, 0.00000000e+00],
        ...,
        [2.86809998e+02, 0.00000000e+00],
        [2.90750000e+02, 0.00000000e+00],
        [2.91290009e+02, 0.00000000e+00]],

       [[2.65730011e+02, 0.00000000e+00],
        [2.65730011e+02, 0.00000000e+00],
        [2.66420013e+02, 0.00000000e+00],
        ...,
        [2.86480011e+02, 0.00000000e+00],
        [2.88799988e+02, 0.00000000e+00],
        [2.90000000e+02, 0.00000000e+00]],

       ...,

       [[2.78880005e+02, 0.00000000e+00],
        [2.78630005e+02, 0.00000000e+00],
        [2.78329987e+02, 0.00000000e+00],
        ...,
        [2.79350006e+02, 8.70740786e-02],
     

In [13]:
image_per_file = 1000
counter = 0
n_files = 1
f = tf.io.TFRecordWriter(f'./images_{n_files}.tfrecord')
for i in range(len(data_matrix)):
    img = data_matrix[i][:, :, 0::3]
    example = tf.train.Example(features=tf.train.Features(feature={'images': tf.train.Feature(float_list=tf.train.FloatList(value=img.reshape(-1))), 
                                                                   'y': tf.train.Feature(float_list=tf.train.FloatList(value=[data_info['Vmax'].values[i]]))}))
    f.write(example.SerializeToString())
    counter += 1
    if counter == image_per_file:
        counter = 0
        print(f'{n_files} record saved')
        f.close()
        n_files += 1
        f = tf.io.TFRecordWriter(f'./images_{n_files}.tfrecord')

1 record saved
2 record saved
3 record saved
4 record saved
5 record saved
6 record saved
7 record saved
8 record saved
9 record saved
10 record saved
11 record saved
12 record saved
13 record saved
14 record saved
15 record saved
16 record saved
17 record saved
18 record saved
19 record saved
20 record saved
21 record saved
22 record saved
23 record saved
24 record saved
25 record saved
26 record saved
27 record saved
28 record saved
29 record saved
30 record saved
31 record saved
32 record saved
33 record saved
34 record saved
35 record saved
36 record saved
37 record saved
38 record saved
39 record saved
40 record saved
41 record saved
42 record saved
43 record saved
44 record saved
45 record saved
46 record saved
47 record saved


In [16]:
data_files = glob.glob('./images_*')

In [19]:
def parse(serialized):
    features={'images':tf.io.FixedLenFeature([80802], tf.float32), 'y': tf.io.FixedLenFeature([1], tf.float32)}
    parsed_example=tf.io.parse_single_example(serialized, features=features)
    image = tf.cast(parsed_example['images'], tf.float32)
    y = tf.cast(parsed_example['y'], tf.float32)
    return tf.reshape(image, (201, 201, 2)), y

In [20]:
dataset = tf.data.TFRecordDataset(data_files).map(parse)
dataset = dataset.shuffle(100)
dataset = dataset.batch(32).prefetch(1)

In [21]:
dataset

<PrefetchDataset shapes: ((None, 201, 201, 2), (None, 1)), types: (tf.float32, tf.float32)>

In [22]:
class Preprocessing(keras.layers.Layer):
    def __init__(self):
        super(Preprocessing, self).__init__()
    def call(self, inputs, training=None):
        if training:
            inputs = tf.image.rot90(inputs, k=np.random.randint(4))
        return tf.image.central_crop(inputs, 0.5)

In [23]:
# input_size = len(X_std)
# output_size = 10

"""
references: 
layers API: https://keras.io/api/layers/ 

1. some parameter tuning: 
    batch size: BATCH_SIZE = 128 #@param ["64", "128", "256", "512"]
    regularizer: l1,l2 
    how to set the initial weight: weights_initializer = keras.initializers.GlorotUniform()  
    batch size: 32 
    how to choose metric?  https://www.tensorflow.org/guide/keras/train_and_evaluate  
        right now, I am using mse. 
    what is callback?  https://keras.io/api/callbacks/  
3. 3 fold cross-validation 
4. hold out data for testing 
5. all years data 
"""



#A function that trains and validates the model and returns the MSE
def train_val_model(train_x,train_y,val_x,val_y, n_epochs, batch_size):
    reg_param = 1e-5
    
    train_X = tf.convert_to_tensor(train_x)
    train_Y = tf.convert_to_tensor(train_y)
    train_X = tf.image.per_image_standardization(train_X)
    
    val_X = tf.convert_to_tensor(val_x)
    val_Y = tf.convert_to_tensor(val_y)
    val_X = tf.image.per_image_standardization(val_X)
    
    weights_initializer = keras.initializers.GlorotUniform()
    

    model = keras.models.Sequential([
        Preprocessing(),
        keras.layers.Conv2D(filters=16, kernel_size=4, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Dense(128, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
        keras.layers.Dense(1, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    ])
    
    #Compiling the model
    model.compile(optimizer=keras.optimizers.Adam(lr=5e-4, beta_1=0.99, beta_2=0.9999),
                  loss='mean_squared_error', #Computes the mean of squares of errors between labels and predictions
                  metrics=['mean_squared_error'], #Computes the mean squared error between y_true and y_pred
                 )
    # initialize TimeStopping callback 
#     time_stopping_callback = tfa.callbacks.TimeStopping(seconds=5*60, verbose=1)
    
    #Training the network
    history = model.fit(train_X,train_Y, 
         epochs=n_epochs,
         batch_size=batch_size, 
         verbose=1,
         validation_split=0.1,
         #callbacks=[tf.keras.callbacks.TensorBoard(run_dir + "/Keras"), time_stopping_callback]
        )
    
    val_score = model.evaluate(val_X, val_Y)
    print("Val Score: ",val_score)
    return history,val_score

In [ ]:
n_epochs=50
batch_size=32
model_history = []  #save the model history in a list after fitting so that we can plot later
val_scores=[]
kf = KFold(n_splits=5)

i=0
for train_index, test_index in kf.split(X_irpmw):
    print("Training on Fold: ",i+1)
    i+=1
    train_x, val_x = X_irpmw[train_index], X_irpmw[test_index]
    train_y, val_y = y[train_index], y[test_index]
    history,val_score = train_val_model(train_x,train_y,val_x,val_y, n_epochs, batch_size)
    model_history.append(history)
    val_scores.append(val_score)
    print("======="*12, end="\n\n\n")

In [30]:
weights_initializer = keras.initializers.GlorotUniform()
reg_param = 1e-5
batch_size=32
model = keras.models.Sequential([
    Preprocessing(),
    keras.layers.Conv2D(filters=16, kernel_size=4, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Dense(128, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
    keras.layers.Dense(1, activation='relu', kernel_initializer = weights_initializer, kernel_regularizer=keras.regularizers.l2(reg_param)),
])

#Compiling the model
model.compile(optimizer=keras.optimizers.Adam(lr=5e-4, beta_1=0.99, beta_2=0.9999),
              loss='mean_squared_error', #Computes the mean of squares of errors between labels and predictions
              metrics=['mean_squared_error'], #Computes the mean squared error between y_true and y_pred
             )
#Training the network
n_epochs=10

history = model.fit(dataset, 
     epochs=n_epochs,
#      verbose=1,
     #callbacks=[tf.keras.callbacks.TensorBoard(run_dir + "/Keras"), time_stopping_callback]
    )

Epoch 1/10
    666/Unknown - 12s 18ms/step - loss: nan - mean_squared_error: 3245.9624

DataLossError: 2 root error(s) found.
  (0) Data loss:  truncated record at 122841460
	 [[node IteratorGetNext (defined at <ipython-input-30-2c7040397189>:25) ]]
  (1) Data loss:  truncated record at 122841460
	 [[node IteratorGetNext (defined at <ipython-input-30-2c7040397189>:25) ]]
	 [[IteratorGetNext/_8]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_5389]

Function call stack:
distributed_function -> distributed_function
